In [10]:
import os
import torch
from PIL import Image
from pathlib import Path

from lavis.models import model_zoo
from lavis.models import load_model_and_preprocess

In [11]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 0 to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())


Device: cuda
Current cuda device: 0
Count of using GPUs: 8


In [12]:
# loads BLIP caption base model, with finetuned checkpoints on MSCOCO captioning dataset.
# this also loads the associated image processors
model, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="large_coco", is_eval=True, device=device)

In [13]:
Dir = Path('/data1/ImageNet1K/Annotations/Data/CLS-LOC/train_1000/')
fileEx = r'.JPEG'

In [14]:
def make_dir_list(dirname):
    dir_list = []
    
    for file in os.listdir(dirname):
        d = os.path.join(dirname, file)
        if os.path.isdir(d):
            dir_list.append(d)
    return dir_list

In [15]:
print(len(make_dir_list(Dir)))

1000


In [16]:
def get_jpeg_list(Dir, file_format):
    JPEG_list = [file for file in os.listdir(Dir) if file.endswith(file_format)]
    return JPEG_list

In [17]:
def make_csv(path):
    f = open("/data1/ImageNet1K/Annotations/Data/CLS-LOC/new_csv/"
     + str(path).split("/")[-1] + ".csv","w",encoding="UTF-8")
    f.write("org_img_path,caption\n")
    
    for file in os.listdir(path):
        w_path = os.path.join(path, file)
        raw_image = Image.open(w_path).convert('RGB')
        
        image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

        # generate caption
        text = model.generate({"image": image})
        
        f.write(w_path + "," + text[0] + "\n")
    f.close()

In [ ]:
for dir in make_dir_list(Dir):
    csv = "/home/datasets/ImageNet1K/ILSVRC/Data/CLS-LOC/train_csv/" + str(dir).split("/")[-1] + ".csv"

    print(dir)
    
    break